In [6]:
import os
import sys
import pycharmm
import pycharmm.psf as psf
import pycharmm.read as read
import pycharmm.write as write
import pycharmm.settings as settings
import pycharmm.lingo as lingo

from pycharmm.lib import charmm as libcharmm

In [9]:
pycharmm.lingo.charmm_script('stream toppar.str')

  
 CHARMM>     stream toppar.str
 VOPEN> Attempting to open::toppar.str::
 OPNLGU> Unit 99 opened for READONLY access to toppar.str

                    INPUT STREAM SWITCHING TO UNIT    99
 RDTITL> * GENERATED BY CHARMM-GUI (HTTP://WWW.CHARMM-GUI.ORG) V3.7 ON AUG, 07. 2023. JOBID=9141926248
 RDTITL> * STREAM FILE FOR TOPOLOGY AND PARAMETER READING
 RDTITL> *
 Parameter: IN1 <- "" <empty>
  
 CHARMM>     
  
 CHARMM>    ! protein topology and parameter
 CHARMM>    open read card unit 10 name toppar/top_all36_prot.rtf
 OPNLGU> Unit already open. The old file will be closed first.
 VCLOSE: Closing unit   10 with status "KEEP"
 VOPEN> Attempting to open::toppar/top_all36_prot.rtf::
 OPNLGU> Unit 10 opened for READONLY access to toppar/top_all36_prot.rtf
  
 CHARMM>    read  rtf card unit 10
 MAINIO> Residue topology file being read from unit  10.
 TITLE> *>>>>>>>>CHARMM36 ALL-HYDROGEN TOPOLOGY FILE FOR PROTEINS <<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<

1

In [15]:
pycharmm.lingo.set_charmm_variable('datfile','ighjval')
datfile= pycharmm.lingo.get_charmm_variable('datfile')
datfile

b'ighjval'